Import Data
- Make a dataframe from the csv
    - There is some inconsistent data (some rows formatted poorly), so we will skip them    


In [2]:
import pandas as pd

FILE = "./data/Assignment-1_Data.csv"

data = pd.read_csv(FILE, sep=";", on_bad_lines="skip", low_memory=False)

print(data.head())

   BillNo                             Itemname  Quantity              Date  \
0  536365   WHITE HANGING HEART T-LIGHT HOLDER         6  01.12.2010 08:26   
1  536365                  WHITE METAL LANTERN         6  01.12.2010 08:26   
2  536365       CREAM CUPID HEARTS COAT HANGER         8  01.12.2010 08:26   
3  536365  KNITTED UNION FLAG HOT WATER BOTTLE         6  01.12.2010 08:26   
4  536365       RED WOOLLY HOTTIE WHITE HEART.         6  01.12.2010 08:26   

  Price  CustomerID         Country  
0  2,55     17850.0  United Kingdom  
1  3,39     17850.0  United Kingdom  
2  2,75     17850.0  United Kingdom  
3  3,39     17850.0  United Kingdom  
4  3,39     17850.0  United Kingdom  


Clean Data
- Only data we care about is BillNo, Itemname, and Country 
    - We drop the other columns
 

In [3]:
columns_to_keep = ['BillNo', 'Itemname', 'Country']

data = data[columns_to_keep]

print(data)

        BillNo                             Itemname         Country
0       536365   WHITE HANGING HEART T-LIGHT HOLDER  United Kingdom
1       536365                  WHITE METAL LANTERN  United Kingdom
2       536365       CREAM CUPID HEARTS COAT HANGER  United Kingdom
3       536365  KNITTED UNION FLAG HOT WATER BOTTLE  United Kingdom
4       536365       RED WOOLLY HOTTIE WHITE HEART.  United Kingdom
...        ...                                  ...             ...
522059  581587          PACK OF 20 SPACEBOY NAPKINS          France
522060  581587          CHILDREN'S APRON DOLLY GIRL          France
522061  581587         CHILDRENS CUTLERY DOLLY GIRL          France
522062  581587      CHILDRENS CUTLERY CIRCUS PARADE          France
522063  581587         BAKING SET 9 PIECE RETROSPOT          France

[522064 rows x 3 columns]


Group Data
- Group transaction data by country

In [6]:
country_datas = {country: data for country, data in data.groupby('Country')}
        
del country_datas["Unspecified"]

for country, df in country_datas.items():
        print(f"Data for {country}:")
        print(df.head())
        print("\n")

Data for Australia:
     BillNo                           Itemname    Country
195  536389       CHRISTMAS LIGHTS 10 REINDEER  Australia
196  536389   VINTAGE UNION JACK CUSHION COVER  Australia
197  536389  VINTAGE HEADS AND TAILS CARD GAME  Australia
198  536389    SET OF 3 COLOURED  FLYING DUCKS  Australia
199  536389         SET OF 3 GOLD FLYING DUCKS  Australia


Data for Austria:
       BillNo                            Itemname  Country
33323  539330  CERAMIC CAKE STAND + HANGING CAKES  Austria
33324  539330  MINI CAKE STAND WITH HANGING CAKES  Austria
33325  539330               JAM JAR WITH PINK LID  Austria
33326  539330  SET3 BOOK BOX GREEN GINGHAM FLOWER  Austria
33327  539330         GREY HEART HOT WATER BOTTLE  Austria


Data for Bahrain:
        BillNo                            Itemname  Country
37241   539500  OCEAN SCENT CANDLE IN JEWELLED BOX  Bahrain
174907  552449  GROW A FLYTRAP OR SUNFLOWER IN TIN  Bahrain
174908  552449          ICE CREAM SUNDAE LIP GLOSS  Bahrai